In [1]:
import sys

import ibis
import ibis.expr.datatypes as dt
import ibis.expr.schema as sch
from kafka import KafkaConsumer
from pyflink.table import EnvironmentSettings, TableEnvironment
import pandas as pd
import json

In [2]:
# rows = []
# consumer = KafkaConsumer("transaction")
# for _, msg in zip(range(10), consumer):
#     rows.append(msg)

In [4]:
# rows[0].value

b'{"trans_date_trans_time": "2024-01-04 00:04:17.624", "cc_num": 4502540000000000000, "merchant": "fraud_Dare-Gibson", "category": "health_fitness", "amt": 8.95, "trans_num": "61b17bfb4709a5ec8b7c249841178db4"}'

In [3]:
# import json

# import pandas as pd

# df = pd.DataFrame([json.loads(row.value) for row in rows])

# df

,trans_date_trans_time,cc_num,merchant,category,amt,trans_num
0,2024-01-05 21:25:01.976,3517810000000000,fraud_Deckow-O'Conner,grocery_pos,138.23,fbd262bbb6d156f57ee4868089051ccf
1,2024-01-05 21:25:02.077,4585130000000,fraud_Erdman-Kertzmann,gas_transport,69.72,6ed5502e8464efee2e341959617bb656
2,2024-01-05 21:25:02.177,676327000000,fraud_Hills-Olson,grocery_net,48.39,a8701a55000472ccae101302e4d2b9f1
3,2024-01-05 21:25:02.277,639024000000,fraud_Huels-Nolan,gas_transport,53.33,10ec7fc2395a5292495c481d9c99e51a
4,2024-01-05 21:25:02.378,2229380000000000,fraud_Hickle Group,shopping_pos,142.88,79bcbb0643cd247e0d207e75ec9e941e
5,2024-01-05 21:25:02.478,213174000000000,"fraud_Schultz, Simonis and Little",grocery_pos,98.01,01264afa6306be16946441c6e5916f3d
6,2024-01-05 21:25:02.578,4826660000000000,fraud_Koepp-Witting,grocery_pos,154.00,170f40b5833c8a9765c5402d8a2a03a2
7,2024-01-05 21:25:02.679,4640400000000,fraud_Koss and Sons,gas_transport,47.19,006c20afb2193760d9f923c065d33b9e
8,2024-01-05 21:25:02.779,4116550000000000,"fraud_Mosciski, Ziemann and Farrell",shopping_net,38.21,f80458b210ec8dd611bbd2790e4011aa
9,2024-01-05 21:25:02.879,3590740000000000,fraud_Stamm-Rodriguez,misc_pos,76.40,22dfa297ab94683757e91ba3b9628282


In [6]:
# df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   trans_date_trans_time  10 non-null     object 
 1   cc_num                 10 non-null     int64  
 2   merchant               10 non-null     object 
 3   category               10 non-null     object 
 4   amt                    10 non-null     float64
 5   trans_num              10 non-null     object 
dtypes: float64(1), int64(1), object(4)
memory usage: 608.0+ bytes


In [3]:
consumer = KafkaConsumer("transaction")
for msg in zip(range(10), consumer):
    print(msg)

(0, ConsumerRecord(topic='transaction', partition=0, offset=6530, timestamp=1704491925924, timestamp_type=0, key=None, value=b'{"trans_date_trans_time": "2024-01-05 21:58:45.924", "cc_num": 180065000000000, "merchant": "fraud_Wisozk and Sons", "category": "misc_pos", "amt": 440.96, "trans_num": "d6ea3690ce1fa61ef02fa6a2915e5f42"}', headers=[], checksum=None, serialized_key_size=-1, serialized_value_size=204, serialized_header_size=-1))
(1, ConsumerRecord(topic='transaction', partition=0, offset=6531, timestamp=1704491926024, timestamp_type=0, key=None, value=b'{"trans_date_trans_time": "2024-01-05 21:58:46.024", "cc_num": 3504180000000000, "merchant": "fraud_DuBuque LLC", "category": "grocery_pos", "amt": 77.22, "trans_num": "2bc0a0318782edce3913b451dda09b5f"}', headers=[], checksum=None, serialized_key_size=-1, serialized_value_size=203, serialized_header_size=-1))
(2, ConsumerRecord(topic='transaction', partition=0, offset=6532, timestamp=1704491926124, timestamp_type=0, key=None, va

In [3]:
local = True

# 1. create a TableEnvironment
env_settings = EnvironmentSettings.in_streaming_mode()
table_env = TableEnvironment.create(env_settings)
# write all the data to one file
table_env.get_config().set("parallelism.default", "1")

# The `flink` backend does not create `TableEnvironment` objects; pass
# the `TableEnvironment` object created above to `ibis.flink.connect`.
connection = ibis.flink.connect(table_env)

# Flink’s streaming connectors aren't part of the binary distribution.
# Link the Kafka connector for cluster execution by adding a JAR file.
connection._exec_sql("ADD JAR 'flink-sql-connector-kafka-3.0.2-1.18.jar'")

In [4]:
source_schema = sch.Schema(
    {
        'trans_date_trans_time': dt.timestamp(scale=3),
        'cc_num': dt.int64,
        'amt': dt.float64,
        'merchant': dt.str,
        'category': dt.str,   
        # 'first': dt.str,
        # 'last': dt.str,
        # 'gender': dt.str,
        # 'street': dt.str,
        # 'city': dt.str,
        # 'state': dt.str,
        # 'zipcode': dt.str,
        # 'latitude': dt.float64,
        # 'longitude': dt.float64,
        # 'city_pop': dt.int32,
        # 'job': dt.str,
        # 'dob': dt.str,
        'trans_num': dt.str,
        # 'unix_time': dt.int64,
        # 'merch_lat': dt.float64,
        # 'merch_long': dt.float64,
        # 'is_fraud': dt.int32,
    }   
)

source_configs = {
    "connector": "kafka",
    "topic": "transaction",
    "properties.bootstrap.servers": "localhost:9092" if local else "kafka:29092",
    "properties.group.id": "test_3",
    "scan.startup.mode": "earliest-offset",
    "format": "json",
}

t = connection.create_table(
    "transaction",
    schema=source_schema,
    tbl_properties=source_configs,
    watermark=ibis.watermark(
        time_col="trans_date_trans_time", allowed_delay=ibis.interval(seconds=15)
    ),
)


In [5]:
# connection.drop_table("transaction")

In [9]:
# 3. create sink Table
sink_schema = sch.Schema(
    {
        'trans_num': dt.str,
        # 'merchant': dt.str,
        # 'category': dt.str,
        # 'cc_num': dt.int64,
        # 'amt': dt.float64,
        'trans_date_trans_time':  dt.timestamp(scale=3),
        'cc_num_amt_5m': dt.float64, 
        'cc_num_amt_60m': dt.float64, 
        # 'merchant_amt_5m': dt.float64, 
        # 'category_amt_5m': dt.float64,       
    }
)

sink_configs = {
    "connector": "kafka",
    "topic": "fraud_features_topic",
    "properties.bootstrap.servers": "localhost:9092" if local else "kafka:29092",
    "format": "json",
}

connection.create_table(
    "fraud_features_table", schema=sink_schema, tbl_properties=sink_configs
)

DatabaseTable: fraud_features_table
  trans_num             string
  trans_date_trans_time timestamp(3)
  cc_num_amt_5m         float64
  cc_num_amt_60m        float64

In [10]:
# connection.drop_table("fraud_features_table")

In [8]:
def create_window_spec(group_by, order_by, interval_in_minutes):
    return ibis.window(
        group_by=group_by,
        order_by=order_by,
        range=(-ibis.interval(minutes=interval_in_minutes), 0),
    )
window_spec = create_window_spec(group_by=t.cc_num, order_by=t.trans_num, interval_in_minutes=5)

In [11]:
# 4. query from source table and perform calculations
agged_cc_num_5m = t[
    t.trans_num,
    t.trans_date_trans_time,
    t.amt.sum()
        .over(create_window_spec(group_by=t.cc_num, order_by=t.trans_date_trans_time, interval_in_minutes=5))
        .name("cc_num_amt_5m")
]

In [12]:
agged_cc_num_60m = t[
    t.trans_num,
    t.trans_date_trans_time,
    t.amt.sum()
        .over(create_window_spec(group_by=t.cc_num, order_by=t.trans_date_trans_time, interval_in_minutes=60))
        .name("cc_num_amt_60m")
]

In [13]:
agged = agged_cc_num_5m.asof_join(
    agged_cc_num_60m, 
    predicates = [
        agged_cc_num_60m['trans_date_trans_time'] <= agged_cc_num_5m['trans_date_trans_time'],
        agged_cc_num_60m['trans_num'] == agged_cc_num_5m['trans_num']
    ]
)

In [14]:
agged = agged[agged.trans_num, agged.trans_date_trans_time, agged.cc_num_amt_5m, agged.cc_num_amt_60m]

In [15]:
agged

r0 := DatabaseTable: transaction
  trans_date_trans_time timestamp(3)
  cc_num                int64
  amt                   float64
  merchant              string
  category              string
  trans_num             string

r1 := Selection[r0]
  selections:
    trans_num:             r0.trans_num
    trans_date_trans_time: r0.trans_date_trans_time
    cc_num_amt_5m:         WindowFunction(func=Sum(r0.amt), frame=RangeWindowFrame(table=r0, 
start=WindowBoundary(value=5 m, preceding=True), end=WindowBoundary(Cast(0, to=interval('m'))), 
group_by=[r0.cc_num], order_by=[asc r0.trans_date_trans_time]))

r2 := Selection[r0]
  selections:
    trans_num:             r0.trans_num
    trans_date_trans_time: r0.trans_date_trans_time
    cc_num_amt_60m:        WindowFunction(func=Sum(r0.amt), frame=RangeWindowFrame(table=r0, 
start=WindowBoundary(value=60 m, preceding=True), end=WindowBoundary(Cast(0, to=interval('m'))), 
group_by=[r0.cc_num], order_by=[asc r0.trans_date_trans_time]))

r3 := AsOfJoin[r1, r2]
  predicates:
    r2.trans_date_trans_time <= r1.trans_date_trans_time
    r2.trans_num == r1.trans_num

r4 := Selection[r3]
  selections:
    trans_num:                   r1.trans_num
    trans_date_trans_time:       r1.trans_date_trans_time
    cc_num_amt_5m:               r1.cc_num_amt_5m
    trans_date_trans_time_right: r2.trans_date_trans_time
    cc_num_amt_60m:              r2.cc_num_amt_60m

Selection[r4]
  selections:
    trans_num:             r4.trans_num
    trans_date_trans_time: r4.trans_date_trans_time
    cc_num_amt_5m:         r4.cc_num_amt_5m
    cc_num_amt_60m:        r4.cc_num_amt_60m

In [16]:
# # 4. query from source table and perform calculations
# agged = t[
#     t.trans_num,
#     t.merchant,
#     t.category,
#     t.cc_num.name("cc_num"),
#     t.amt,
#     t.amt.sum()
#         .over(create_window_spec(group_by=t.cc_num, order_by=t.trans_date_trans_time, interval_in_minutes=1))
#         .name("cc_num_amt_5m"),
#     # t.amt.sum()
#     #     .over(create_window_spec(group_by=t.cc_num, order_by=t.trans_date_trans_time, interval_in_minutes=5))
#     #     .name("merchant_amt_5m"),
#     # t.amt.sum()
#     #     .over(create_window_spec(group_by=t.cc_num, order_by=t.trans_date_trans_time, interval_in_minutes=10))
#     #     .name("category_amt_5m"),
   
# ]

In [17]:
agged

r0 := DatabaseTable: transaction
  trans_date_trans_time timestamp(3)
  cc_num                int64
  amt                   float64
  merchant              string
  category              string
  trans_num             string

r1 := Selection[r0]
  selections:
    trans_num:             r0.trans_num
    trans_date_trans_time: r0.trans_date_trans_time
    cc_num_amt_5m:         WindowFunction(func=Sum(r0.amt), frame=RangeWindowFrame(table=r0, 
start=WindowBoundary(value=5 m, preceding=True), end=WindowBoundary(Cast(0, to=interval('m'))), 
group_by=[r0.cc_num], order_by=[asc r0.trans_date_trans_time]))

r2 := Selection[r0]
  selections:
    trans_num:             r0.trans_num
    trans_date_trans_time: r0.trans_date_trans_time
    cc_num_amt_60m:        WindowFunction(func=Sum(r0.amt), frame=RangeWindowFrame(table=r0, 
start=WindowBoundary(value=60 m, preceding=True), end=WindowBoundary(Cast(0, to=interval('m'))), 
group_by=[r0.cc_num], order_by=[asc r0.trans_date_trans_time]))

r3 := AsOfJoin[r1, r2]
  predicates:
    r2.trans_date_trans_time <= r1.trans_date_trans_time
    r2.trans_num == r1.trans_num

r4 := Selection[r3]
  selections:
    trans_num:                   r1.trans_num
    trans_date_trans_time:       r1.trans_date_trans_time
    cc_num_amt_5m:               r1.cc_num_amt_5m
    trans_date_trans_time_right: r2.trans_date_trans_time
    cc_num_amt_60m:              r2.cc_num_amt_60m

Selection[r4]
  selections:
    trans_num:             r4.trans_num
    trans_date_trans_time: r4.trans_date_trans_time
    cc_num_amt_5m:         r4.cc_num_amt_5m
    cc_num_amt_60m:        r4.cc_num_amt_60m

In [18]:
# 5. emit query result to sink table
connection.insert("fraud_features_table", agged)

op = <ibis.expr.operations.relations.DatabaseTable object at 0x7fc9f7a01300>, table_str = transaction t3
op = <ibis.expr.operations.relations.DatabaseTable object at 0x7fc9f7a01d20>, table_str = transaction t3
op = <ibis.expr.operations.relations.Selection object at 0x7fc9f79d67a0>, table_str = t0


ValueError: not enough values to unpack (expected 2, got 1)

In [15]:
if local:
    # Use the Kafka Python client to stream records from the sink topic.
    # Otherwise, the mini cluster will shut down upon script completion.
    consumer = KafkaConsumer("fraud_features_topic")
    for _, msg in zip(range(10), consumer):
        print(msg)


ConsumerRecord(topic='fraud_features_topic', partition=0, offset=9686, timestamp=1704326690514, timestamp_type=0, key=None, value=b'{"trans_num":"f3870d607e888d38a79ef952b28e676b","merchant":"fraud_Lubowitz-Walter","category":"kids_pets","cc_num":4900630000000,"amt":39.64,"cc_num_amt_5m":39.64}', headers=[], checksum=None, serialized_key_size=-1, serialized_value_size=163, serialized_header_size=-1)
ConsumerRecord(topic='fraud_features_topic', partition=0, offset=9687, timestamp=1704326690514, timestamp_type=0, key=None, value=b'{"trans_num":"1887c447c6d598e2559e1b1f3f35d5f4","merchant":"fraud_Bednar PLC","category":"kids_pets","cc_num":2383460000000000,"amt":89.61,"cc_num_amt_5m":89.61}', headers=[], checksum=None, serialized_key_size=-1, serialized_value_size=161, serialized_header_size=-1)
ConsumerRecord(topic='fraud_features_topic', partition=0, offset=9688, timestamp=1704326690514, timestamp_type=0, key=None, value=b'{"trans_num":"415e39baad9608d29025e28ed99dda98","merchant":"frau